<a href="https://colab.research.google.com/github/SubMishMar/rotation_averaging/blob/main/single_rotation_averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:
np.random.seed(42)  # reproducibility

In [3]:
# Utility
def add_rotation_noise(R_true, noise_level_deg=5.0):
    axis = np.random.randn(3)
    axis /= np.linalg.norm(axis)
    angle = np.random.normal(0, np.deg2rad(noise_level_deg))
    R_noise = R.from_rotvec(axis * angle).as_matrix()
    return R_true.as_matrix() @ R_noise


def hat(w):
    wx, wy, wz = w
    return np.array([[   0, -wz,  wy],
                     [  wz,   0, -wx],
                     [ -wy,  wx,   0]], dtype=float)

def vee(S):
    return np.array([S[2,1], S[0,2], S[1,0]], dtype=float)

def exponential_map(w, eps=1e-8):
    theta = np.linalg.norm(w)
    W = hat(w)
    if theta < eps:
        # A ~ 1 - θ^2/6, B ~ 1/2 - θ^2/24
        A = 1.0 - (theta**2)/6.0
        B = 0.5 - (theta**2)/24.0
    else:
        A = np.sin(theta) / theta
        B = (1.0 - np.cos(theta)) / (theta**2)
    return np.eye(3) + A * W + B * (W @ W)


def logarithm_map(R, eps=1e-8):
    tr = np.trace(R)
    cos_phi = (tr - 1.0) * 0.5
    cos_phi = np.clip(cos_phi, -1.0, 1.0)
    phi = np.arccos(cos_phi)

    if phi < eps:
        return 0.5 * vee(R - R.T)

    if np.pi - phi < 1e-5:
        Rp = (R + np.eye(3)) * 0.5
        idx = np.argmax(np.diag(Rp))
        v = np.zeros(3)
        v[idx] = np.sqrt(max(Rp[idx, idx], 0.0))
        j = (idx + 1) % 3
        k = (idx + 2) % 3
        if v[idx] > eps:
            v[j] = Rp[j, idx] / v[idx]
            v[k] = Rp[k, idx] / v[idx]
        v = v / (np.linalg.norm(v) + eps)
        return phi * v

    # Generic case
    return (phi / (2.0 * np.sin(phi))) * vee(R - R.T)

def right_jacobian(v, eps=1e-8):
    theta = np.linalg.norm(v)
    V = hat(v)
    if theta < eps:
        return np.eye(3) - 0.5*V + (1.0/12.0)*(V @ V)
    A = (1.0 - np.cos(theta)) / (theta**2)
    B = (theta - np.sin(theta)) / (theta**3)
    return np.eye(3) - A*V + B*(V @ V)

def right_jacobian_inverse(v, eps=1e-8):
    theta = np.linalg.norm(v)
    V = hat(v)
    if theta < eps:
        return np.eye(3) + 0.5*V + (1.0/12.0)*(V @ V)
    cot_half = 1.0 / np.tan(0.5 * theta)
    B = (1.0 / (theta**2)) - (0.5 / theta) * cot_half
    return np.eye(3) + 0.5*V + B*(V @ V)

In [6]:
# Minimize Chordal Difference
def chordal_average(rotation_matrices):
    # Computes the chordal average of a set of rotation matrices.
    # This is the minimizer when rotations are assumed to be drawn from Langevin
    # Distribution.

    # Step 1: Average the rotation matrices.
    M = np.zeros((3, 3))
    for Rn in rotation_matrices:
        M += Rn
    M /= len(rotation_matrices)
    # Step 2: Project the average matrix onto SO(3).
    U, _, Vt = np.linalg.svd(M)
    R = U @ Vt
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = U @ Vt
    return R

In [7]:
# Minimize Geodesic difference
# Minimization of Geodesic difference results when rotations are assumed to be
# sampled from a wrapped Gaussian distribution over Lie groups
def residuals_and_jacobians(R_est, R_meas_list):
    rows = []
    jacobians = []
    Rt = R_est.T
    for Rm in R_meas_list:
        r_i = logarithm_map(Rt @ Rm)
        jacobian_i = -right_jacobian_inverse(r_i)   # <-- removed @ Rm.T @ R_est
        rows.append(r_i.reshape(-1, 1))
        jacobians.append(jacobian_i)
    return np.vstack(rows), np.vstack(jacobians)

def cost(residual):
    return 0.5 * float(residual.T @ residual)

def gauss_newton(R_meas_list, R_init,
                 max_iters=500, f_tol=1e-8, x_tol=1e-10):
    R_est = R_init
    history = []
    step = 0.0
    for k in range(max_iters):
        r, J = residuals_and_jacobians(R_est, R_meas_list)
        f = cost(r)

        history.append((k, f, 'Gauss Newton', step))

        H = J.T @ J
        g = J.T @ r
        delta = (-np.linalg.inv(H) @ g).reshape(3)   # <-- flatten to (3,)

        R_new = R_est @ exponential_map(delta)

        step = float(np.linalg.norm(delta))
        r_new, _ = residuals_and_jacobians(R_new, R_meas_list)
        f_new = cost(r_new)

        if abs(f_new - f) < f_tol or step < x_tol:
            R_est = R_new
            break

        R_est = R_new

    return R_est, history

In [8]:
# Generate a True Rotation Value
angle_x = 30.0  # degrees
angle_y = 20.0
angle_z = 10.0
euler_order = "xyz"
true_euler_deg = {
    'x': angle_x,
    'y': angle_y,
    'z': angle_z
}
angles = [true_euler_deg[axis] for axis in euler_order]
R_true = R.from_euler(euler_order, angles, degrees=True)

# Generate N noisy versions
N = 100
noisy_rotations = [add_rotation_noise(R_true, noise_level_deg=2.5) for _ in range(N)]
R_init = add_rotation_noise(R_true, noise_level_deg=10.0)
for i, Rn in enumerate(noisy_rotations):
  rot_vec = R.from_matrix(Rn).as_euler('xyz', degrees=True)
  print(f"\nNoisy Rotation {i+1}:\n{rot_vec}")

R_estimated = chordal_average(noisy_rotations)

print('\nR_true: \n', R_true.as_euler('xyz', degrees=True))
print('R_estimated: \n', R.from_matrix(R_estimated).as_euler('xyz', degrees=True))


Noisy Rotation 1:
[33.04297494 17.90030362 12.33309924]

Noisy Rotation 2:
[30.24461014 18.81677465 11.5738735 ]

Noisy Rotation 3:
[30.70077043 19.04256262 10.18188317]

Noisy Rotation 4:
[30.35925509 20.42749252 11.42014662]

Noisy Rotation 5:
[33.09440195 18.12125431 11.63041493]

Noisy Rotation 6:
[26.52530857 20.55378556 10.12130432]

Noisy Rotation 7:
[29.34374475 20.48956334  9.25967239]

Noisy Rotation 8:
[25.64724339 20.11788652  6.3350751 ]

Noisy Rotation 9:
[29.86457551 23.02272254  9.5469786 ]

Noisy Rotation 10:
[29.88371337 19.78562406  9.53112918]

Noisy Rotation 11:
[29.28071281 19.79765326 10.01645702]

Noisy Rotation 12:
[27.29701828 19.36417389  8.76845555]

Noisy Rotation 13:
[29.93672233 20.89097246 10.33769757]

Noisy Rotation 14:
[29.59360816 20.03125298 12.1573666 ]

Noisy Rotation 15:
[27.9802494  18.89788577 10.37934982]

Noisy Rotation 16:
[32.08090389 18.9912581  12.71408107]

Noisy Rotation 17:
[31.67266085 21.90453662 11.07502927]

Noisy Rotation 18:
[31

In [9]:
R_est, history = gauss_newton(noisy_rotations, R_init)

/tmp/ipython-input-2008422853.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 0.5 * float(residual.T @ residual)


In [10]:
print('\nR_true: \n', R_true.as_euler('xyz', degrees=True))
print('R_init: \n', R.from_matrix(R_init).as_euler('xyz', degrees=True))
print('R_estimated: \n', R.from_matrix(R_est).as_euler('xyz', degrees=True))


R_true: 
 [30. 20. 10.]
R_init: 
 [29.53094286 19.85577976  9.91403861]
R_estimated: 
 [30.02437535 20.17288967 10.07282972]
